In [71]:
using Plots
using ProgressMeter
gr()

Plots.GRBackend()

In [2]:
struct rule
    number_dictionary       #Deprecated, old implementation was running each element over a dictionary.
                            #Too overkill for simple integer systems. Necessary for complex integer systems.
                            #Doesn't work for continuous numbers.
    filter::Array{Int64,2}  #The filter passed through. Determines which neighbors are considered.
end

In [95]:
function Cellular(rule,steps=100,startarr=0,randomize=true,dim=(100,100))  #Main function
    #=
    rule : rule structure. See above
    startarr : starting array.
    randomize : Whether or not to randomize the initial array. By default 100x100. If true, will ignore the value of startarr.
    =#
    
    randomize ? Grid=rand(0:1,100,100) : Grid=startarr  #randomize initial array if necessary.
    
    heatmap(Grid,c=:gray)                      #Heatmap setup.
    p=Progress(100,1)                          #Progress bar setup.
    animation= @animate for i=1:steps          #Loop, @animate for gif.
        Grid_temp=conv2(Grid,rule.filter)      #Running the filter over the array.
        
        Grid1=map(twoorthree,Grid_temp[2:101,2:101])  #Checks whether there are two or three near neighbors. Rules 1,2,3.
        Grid2=map(threeornone,Grid_temp[2:101,2:101]) #Checks whether there are three near neighbors.
        Grid_inv=map(invers,Grid)                     #Inverses the initial array (1 -> 0, 0->1)
        
        Grid = (Grid .& Grid1 ).| (Grid_inv .& Grid2) # ( TwoOrThree AND Living) OR ( Three AND Dead)
        
        
        heatmap!(Grid,c=:gray)                      #Next frame
        next!(p)       #Go through the progress bar
        end 
    gif(animation,"tmp1.gif",fps=10)  #Animate at 10 fps.
end
        

Cellular (generic function with 5 methods)

The performance of the Cellular function is **massively** dependant on the performance of the histogram package. And it's bad. Python is even worse due to MatPlotLib prioritizing beauty over performance. Using the MatPlotLib backend on this code makes it run about twice slower. 

In [42]:
function twoorthree(x)   #Straight Forward.
    if x==2 | x==3
        return 1
    else
        return 0
    end
end

twoorthree (generic function with 1 method)

In [38]:
R=rule(Dict([(0=>0),(1=>0),(2=>1),(3=>1),(4=>0),(5=>0),(6=>0),(7=>0),(8=>0)]),[1 1 1;1 0 1;1 1 1]) #Ignore the dictionary.

rule(Dict(0=>0,4=>0,7=>0,2=>1,3=>1,5=>0,8=>0,6=>0,1=>0), [1 1 1; 1 0 1; 1 1 1])

In [28]:
function threeornone(x)  #Straight Forward as well.
    if x==3
        return 1
    else
        return 0
    end
end

threeornone (generic function with 1 method)

In [34]:
function invers(x)  #Same
    if x==0
        return 1
    else
        return 0
    end
end

invers (generic function with 1 method)

In [97]:
@time anim=Cellular(R)  #~6 minutes on a i5-6500. Memory isn't an issue. Skipping frames makes it much faster.
                        #Ignore the innocent warnings.

Progress:   4%|██                                       |  ETA: 0:00:39WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:   6%|██                                       |  ETA: 0:00:48WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:   8%|███                                      |  ETA: 0:00:53WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  10%|████                                     |  ETA: 0:00:58WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  12%|█████                                    |  ETA: 0:01:02WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  13%|█████                                    |  ETA: 0:01:04WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  14%|██████                                   |  ETA: 

Progress:  45%|██████████████████                       |  ETA: 0:01:38WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  46%|███████████████████                      |  ETA: 0:01:38WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  47%|███████████████████                      |  ETA: 0:01:37WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  48%|████████████████████                     |  ETA: 0:01:37WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  49%|████████████████████                     |  ETA: 0:01:37WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  50%|████████████████████                     |  ETA: 0:01:37WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  51%|█████████████████████                    |  ETA: 

Progress:  89%|████████████████████████████████████     |  ETA: 0:00:35WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  90%|█████████████████████████████████████    |  ETA: 0:00:32WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  91%|█████████████████████████████████████    |  ETA: 0:00:29WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  92%|██████████████████████████████████████   |  ETA: 0:00:26WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  93%|██████████████████████████████████████   |  ETA: 0:00:23WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  94%|███████████████████████████████████████  |  ETA: 0:00:20WARNING: gray is found in more than one library: cmocean, colorcet. Choosing cmocean
Progress:  95%|███████████████████████████████████████  |  ETA: 

381.824817 seconds (328.31 M allocations: 10.467 GiB, 0.42% gc time)


INFO: Saved animation to /home/qflex/jlfiles/tmp1.gif


Plots.AnimatedGif("/home/qflex/jlfiles/tmp1.gif")